In [13]:
import pandas as pd
import sys
import csv

In [14]:
batchsizes = [1,2,4,8,16]

In [15]:
data = pd.read_csv('retry_all_ssr_results.csv')

In [16]:
print(set(data['project'].tolist()))

{'sonarqube', 'loomio', 'concerto', 'vagrant', 'rails', 'cloudify', 'fog', 'graylog2-server', 'jruby', 'cloud_controller_ng', 'puppet', 'opal', 'gradle', 'open-build-service', 'geoserver', 'rubinius', 'metasploit-framework', 'orbeon-forms', 'sufia'}


In [17]:
data['project_reqd_builds'] = pd.to_numeric(data['project_reqd_builds'], downcast="float", errors='coerce')
data["project_missed_builds"] = pd.to_numeric(data["project_missed_builds"], downcast="float", errors='coerce')
data["project_saved_builds"] = pd.to_numeric(data["project_saved_builds"], downcast="float", errors='coerce')
data['batch_delays'] = pd.to_numeric(data["batch_delays"], downcast="float", errors='coerce')

In [18]:
delay_avg = []
delays = data['project_delays'].tolist()

for i in delays:
    if i == '[]':
        delay_avg.append(0)
    else:
        row = i[1:-1].split(', ')
        row = [int(x) for x in row]
        delay_avg.append(sum(row))
data['total_delay'] = delay_avg

In [19]:
# delay_avg = []
# delays = data['batch_delays'].tolist()

# for i in delays:
#     if i == '[]':
#         delay_avg.append(0)
#     else:
#         row = [x for x in list(i)[1:-1] if (x != ',' and x != ' ')]
#         row = [int(x) for x in row]
#         delay_avg.append(sum(row))
data['total_batch_delay'] = data['batch_delays'].tolist()

In [20]:
project_list = set(data['project'].tolist())
batching_alg = set(data['algorithm'].tolist())

In [21]:
data

,project,algorithm,batch_size,confidence,project_reqd_builds,project_missed_builds,project_build_duration,project_saved_builds,project_delays,testall_size,batch_delays,total_delay,total_batch_delay
0,rails,BATCHBISECT,1,2,52.403522,6.838185,9090954,47.596478,"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...",1477,0.0,202,0.0
1,rails,BATCHBISECT,1,3,36.763710,6.635071,6314992,63.236290,"[3, 3, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 2, 3, ...",1477,0.0,248,0.0
2,rails,BATCHBISECT,1,4,27.826675,8.124577,4845255,72.173325,"[4, 3, 3, 3, 3, 4, 2, 2, 2, 4, 3, 3, 3, 3, 4, ...",1477,0.0,356,0.0
3,rails,BATCHBISECT,1,5,23.358158,7.515234,3948572,76.641838,"[2, 5, 5, 4, 4, 5, 3, 3, 4, 5, 3, 5, 3, 2, 4, ...",1477,0.0,391,0.0
4,rails,BATCHBISECT,1,6,19.363575,8.869329,3347533,80.636429,"[3, 6, 2, 2, 5, 2, 4, 2, 6, 3, 3, 3, 6, 3, 6, ...",1477,0.0,531,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3244,graylog2-server,BATCHSTOP4,16,16,14.214464,0.748130,61179,41.396507,"[5, 10, 22]",401,1746.0,37,1746.0
3245,graylog2-server,BATCHSTOP4,16,17,9.476309,0.997506,48515,44.139652,"[7, 14, 3, 6]",401,1680.0,30,1680.0
3246,graylog2-server,BATCHSTOP4,16,18,10.473816,0.997506,46096,46.882793,"[9, 18, 8, 22]",401,1575.0,57,1575.0
3247,graylog2-server,BATCHSTOP4,16,19,9.476309,1.246883,33907,49.625935,"[11, 19, 13, 3, 22]",401,1495.0,68,1495.0


In [22]:
result_file = open('retry_ssr_results.csv', 'w')
res_headers = ['project', 'method', 'algorithm', 'batch_size', 'builds_reqd', 'batch_delays', 'sbs_delays', 'confidence']
res_writer = csv.writer(result_file)
res_writer.writerow(res_headers)

84

In [23]:
print(len(project_list))

19


In [24]:
for p in project_list:
    p_name = p + '.csv'
    alg_scores = {}

    p_data = data[data['project'] == p]
    #print(p_data)
    for alg in batching_alg:

        alg_data = p_data[p_data['algorithm'] == alg]
        #print(alg_data)

        for b in batchsizes:

            if alg == 'BATCH4':
                if b != 4:
                    continue

            if alg == 'BATCHSTOP4':
                if b < 4:
                    continue

            batch_data = alg_data[ alg_data['batch_size'] == b]

            reqd_list = batch_data['project_reqd_builds'].tolist()
            delay_list = batch_data['total_delay'].tolist()
            batch_delay_list = batch_data['total_batch_delay'].tolist()
            min_distance = sys.maxsize

            best = [p_name, 'ssr', alg, b, 0, 0, 0, 0]
            best_i = 0

            for i in range(len(reqd_list)):
                distance = ((reqd_list[i]**2)+((delay_list[i]+batch_delay_list[i])**2))**0.5
                if distance < min_distance:
                    best[4] = reqd_list[i]
                    best[5] = batch_delay_list[i]
                    best[6] = delay_list[i]
                    best[7] = i+2
                    best_i = i
                    min_distance = distance

            res_writer.writerow(best)
            print(best)

['sonarqube.csv', 'ssr', 'BATCH4', 4, 18.33333396911621, 157.0, 92, 12]
['sonarqube.csv', 'ssr', 'BATCHSTOP4', 4, 18.33333396911621, 157.0, 92, 12]
['sonarqube.csv', 'ssr', 'BATCHSTOP4', 8, 30.66666603088379, 547.0, 77, 18]
['sonarqube.csv', 'ssr', 'BATCHSTOP4', 16, 38.66666793823242, 1588.0, 66, 18]
['sonarqube.csv', 'ssr', 'BATCHBISECT', 1, 35.66666793823242, 0.0, 38, 3]
['sonarqube.csv', 'ssr', 'BATCHBISECT', 2, 25.0, 57.0, 43, 5]
['sonarqube.csv', 'ssr', 'BATCHBISECT', 4, 19.0, 157.0, 92, 12]
['sonarqube.csv', 'ssr', 'BATCHBISECT', 8, 30.66666603088379, 547.0, 77, 18]
['sonarqube.csv', 'ssr', 'BATCHBISECT', 16, 38.66666793823242, 1588.0, 66, 18]
['loomio.csv', 'ssr', 'BATCH4', 4, 43.90243911743164, 180.0, 31, 7]
['loomio.csv', 'ssr', 'BATCHSTOP4', 4, 43.90243911743164, 180.0, 31, 7]
['loomio.csv', 'ssr', 'BATCHSTOP4', 8, 41.46341323852539, 420.0, 71, 19]
['loomio.csv', 'ssr', 'BATCHSTOP4', 16, 66.34146118164062, 1203.0, 29, 15]
['loomio.csv', 'ssr', 'BATCHBISECT', 1, 37.07316970825